In [13]:

import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as sm
from utils.machine_learning import one_hot_decoder
from utils.machine_learning.confusion import ConfusionEstimator


In [2]:
mne.set_log_level('CRITICAL')

In [3]:
content_root = '../'
subjects_folder_path = os.path.join(content_root, 'Source/Subjects')
subject_path = os.path.join(subjects_folder_path, 'Ga_Fed_06')
info_path = os.path.join(subject_path, 'Info',
                        'ML_Subject05_P1_tsss_mc_trans_info.pkl')
resp_lock_lm_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_LM_B1_epochs.fif')
resp_lock_rm_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_RM_B1_epochs.fif')
resp_lock_li_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_LI_B1_epochs.fif')
resp_lock_ri_B1_epochs_path = os.path.join(
    subject_path, 'Epochs', 'RespCor_RI_B1_epochs.fif')

resp_lock_lm_B1_epochs = mne.read_epochs(resp_lock_lm_B1_epochs_path)
resp_lock_li_B1_epochs = mne.read_epochs(resp_lock_li_B1_epochs_path)
resp_lock_rm_B1_epochs = mne.read_epochs(resp_lock_rm_B1_epochs_path)
resp_lock_ri_B1_epochs = mne.read_epochs(resp_lock_ri_B1_epochs_path)

resp_lock_li_B1_epochs.resample(200)
resp_lock_lm_B1_epochs.resample(200)
resp_lock_ri_B1_epochs.resample(200)
resp_lock_rm_B1_epochs.resample(200)

combiner = EpochsCombiner(
    resp_lock_lm_B1_epochs.copy(),
    resp_lock_li_B1_epochs.copy(),
    resp_lock_rm_B1_epochs.copy(),
    resp_lock_ri_B1_epochs.copy()
)
first_class_indices = (0, 1)
second_class_indices = (2, 3)
combiner.combine(first_class_indices, second_class_indices, shuffle=True)

In [6]:

savepath = '../Source/Subjects/Az_Mar_05/TFR/left_vs_right/'
project_name = 'fake_name'

import_opt = dict(
        savepath=savepath+'/',
        out_name=project_name,
        fs=200,
        input_type='trials',
        target_type='int',
        picks={'meg': 'grad'},
        scale=True,
        crop_baseline=True,
        decimate=None,
        scale_interval=(0, 60),
        n_folds=5,
        overwrite=True,
        segment=False,
        test_set='holdout'
    )
meta = mf.produce_tfrecords((combiner.X, combiner.Y), **import_opt)
dataset = mf.Dataset(meta, train_batch=100)
lf_params = dict(
        n_latent=32,
        filter_length=50,
        nonlin=tf.keras.activations.elu,
        padding='SAME',
        pooling=10,
        stride=10,
        pool_type='max',
        model_path=import_opt['savepath'],
        dropout=.4,
        l2_scope=["weights"],
        l2=1e-6
)

model = mf.models.LFCNN(dataset, lf_params)
model.build()

importing from tuple
input shapes: X- (208, 204, 200) targets- (208, 1)
Preprocessing:
Scaling to interval 0.0 - 60.0
Splitting sets
Preprocessed: (208, 1, 200, 204) (208, 2) folds: 6 x 34
Prepocessed sample shape: (1, 200, 204)
Target shape actual/metadata:  (2,) (2,)
Saving TFRecord# 0


2022-02-21 17:38:28.277725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-21 17:38:28.277761: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-21 17:38:28.277777: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arcolinux-machine): /proc/driver/nvidia/version does not exist
2022-02-21 17:38:28.278108: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Setting reg for dmx, to l2
Built: dmx input: (None, 1, 200, 204)
Setting reg for tconv, to l2
Built: tconv input: (None, 1, 200, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 20, 32)
Input shape: (1, 200, 204)
y_pred: (None, 2)
Initialization complete!


In [7]:
weights_path = os.path.join(subject_path, 'Weights', 'LM&LI_vs_RM&RI_B1-B8.h5')
model.km.load_weights(weights_path)

In [8]:
loss, acc = model.evaluate(model.dataset.val)
print(acc)

1/1 [==============================] - 0s 388ms/step - loss: 0.7108 - cat_ACC: 0.6897
0.6896551847457886


In [20]:
y_t_val, y_p_val = model.predict(model.dataset.val)

print(
    'val acc',
    sm.accuracy_score(
        one_hot_decoder(y_t_val),
        one_hot_decoder(y_p_val),
    )
)

tn, fp, fn, tp = sm.confusion_matrix(one_hot_decoder(y_t_val), one_hot_decoder(y_p_val)).ravel()
ce = ConfusionEstimator(tp, tn, fp, fn)

print(
    f'\tT\tF\n'\
    f'P\t{tp}\t{fp}\n'\
    f'N\t{tn}\t{fn}\n'
    f'Accuracy: {ce.acc}\n'\
    f'Specificity: {ce.spec}\n'\
    f'Sensitivity: {ce.sens}'
)
y_t_train, y_p_train = model.predict(model.dataset.train)

print(
    'train acc',
    sm.accuracy_score(
        one_hot_decoder(y_t_train),
        one_hot_decoder(y_p_train),
    )
)

tn, fp, fn, tp = sm.confusion_matrix(one_hot_decoder(y_t_train), one_hot_decoder(y_p_train)).ravel()
ce = ConfusionEstimator(tp, tn, fp, fn)

print(
    f'\tT\tF\n'\
    f'P\t{tp}\t{fp}\n'\
    f'N\t{tn}\t{fn}\n'
    f'Accuracy: {ce.acc}\n'\
    f'Specificity: {ce.spec}\n'\
    f'Sensitivity: {ce.sens}'
)

y_t = np.concatenate([y_t_val, y_t_train], axis=0)
y_p = np.concatenate([y_p_val, y_p_train], axis=0)


print(
    'total acc',
    sm.accuracy_score(
        one_hot_decoder(y_t),
        one_hot_decoder(y_p),
    )
)
tn, fp, fn, tp = sm.confusion_matrix(one_hot_decoder(y_t), one_hot_decoder(y_p)).ravel()
ce = ConfusionEstimator(tp, tn, fp, fn)

print(
    f'\tT\tF\n'\
    f'P\t{tp}\t{fp}\n'\
    f'N\t{tn}\t{fn}\n'
    f'Accuracy: {ce.acc}\n'\
    f'Specificity: {ce.spec}\n'\
    f'Sensitivity: {ce.sens}'
)


val acc 0.41379310344827586
	T	F
P	9	13
N	3	4
Accuracy: 0.41379310344827586
Specificity: 0.1875
Sensitivity: 0.6923076923076923
train acc 0.52
	T	F
P	42	32
N	10	16
Accuracy: 0.52
Specificity: 0.23809523809523808
Sensitivity: 0.7241379310344828
total acc 0.49612403100775193
	T	F
P	51	45
N	13	20
Accuracy: 0.49612403100775193
Specificity: 0.22413793103448276
Sensitivity: 0.7183098591549296


In [21]:
X = combiner.X.copy()
print(X.shape)
X = np.transpose(np.expand_dims(X, axis = 1), (0, 1, 3, 2))
print(X.shape)

(208, 204, 200)
(208, 1, 200, 204)


In [19]:
model.predict((combiner.X, combiner.Y))

2022-02-21 14:02:45.894793: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 407347200 exceeds 10% of free system memory.
2022-02-21 14:02:46.123099: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 135782400 exceeds 10% of free system memory.
2022-02-21 14:02:46.855018: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 135782400 exceeds 10% of free system memory.


ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [25]:
models_path = '/home/user/Projects/FingerMovementDecoder/Source/Subjects/Ga_Fed_06/Models'
model = tf.keras.models.load_model(
    os.path.join(models_path, 'LM&LI_vs_RM&RI_B1-B8.h5'),
    custom_objects={
        'DeMixing': mf.layers.DeMixing,
        'LFTConv': mf.layers.LFTConv,
        'TempPooling': mf.layers.TempPooling,
        'Dense': mf.layers.Dense,
        'identity': tf.identity,
        'softmax_cross_entropy_with_logits_v2': tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2,
    }
)

Built: dmx input: (None, 1, 200, 204)
Built: tconv input: (None, 1, 200, 32)
Built: fc input: (None, 1, 20, 32)


In [26]:
model(X)

2022-02-21 20:40:56.771275: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 67891200 exceeds 10% of free system memory.


<tf.Tensor: shape=(208, 2), dtype=float32, numpy=
array([[-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.0509114],
       [-0.0070357, -0.050